In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,sum,avg,max

In [0]:
#Creating the schema for the Orders csv files
order_schema = StructType([
    StructField("O_ORDERKEY", IntegerType()),
    StructField("O_CUSTKEY", StringType()),
    StructField("O_ORDERSTATUS", StringType()),
    StructField("O_TOTALPRICE", DoubleType()),
    StructField("O_ORDERDATE", DateType()),
    StructField("O_ORDERPRIORITY", StringType()),
    StructField("O_CLERK", StringType()),
    StructField("O_SHIPPRIORITY", IntegerType()),
    StructField("O_COMMENT", StringType())
])


In [0]:
#Creating the schema for the Customer csv files
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DoubleType()),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])


In [0]:
#Creating the Orders dataframe by specifying the schema 
df_ord_sch= spark.read.format("csv").option("header",True).schema(order_schema).load("/mnt/Gen2/Orders/csvFiles/")

In [0]:
#Creating the dataframe by specifying the schema using .schema option. We don't see any DAG getting created with schema is specified
df_cust_sch= spark.read.format("csv").option("header",True).schema(cust_schema).load("/mnt/Gen2/Customer/csvFiles/")

In [0]:
# check for parameter value and the default value is 10 MB
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

Out[8]: '10485760b'

In [0]:
#Setting to 2 MB
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",2194304)

In [0]:
# Check the execution plan and you will find sort merge join being used 
df_ord_sch.join(df_cust_sch, df_ord_sch.O_CUSTKEY == df_cust_sch.C_CUSTKEY, "inner").count()

Out[10]: 100002

In [0]:
df_ord_sch.join(df_cust_sch.hint("broadcast"), df_ord_sch.O_CUSTKEY == df_cust_sch.C_CUSTKEY, "inner").count()

Out[11]: 100002

In [0]:
df_ord_sch.join(df_cust_sch.hint("merge"), df_ord_sch.O_CUSTKEY == df_cust_sch.C_CUSTKEY, "inner").count()

Out[12]: 100002

In [0]:
display(df_ord_sch.limit(3))

O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT
954213,149285,O,215440.73,1998-06-13,2-HIGH,Clerk#000000796,0,furiously special f
1407841,79810,F,175384.14,1995-01-06,1-URGENT,Clerk#000000526,0,furiously special f
1330759,80399,F,208297.39,1992-04-15,2-HIGH,Clerk#000000592,0,furiously special f
